# Lecture 1 — Ejercicios de procesado de imagen digital
**Fecha:** 2025-10-06

Este cuaderno acompaña a la lectura *Introducción al procesado de imagen digital*.  
Trabajaremos con imágenes **sintéticas** generadas con `numpy` para que el cuaderno sea 100% reproducible sin conexión.

**Objetivos:**
- Comprender la representación numérica de una imagen (matrices y tensores).
- Practicar indexación, *slicing*, trasposición e inversiones.
- Manipular canales RGB y normalización de intensidades.
- Entender los formatos HWC y CHW y calcular posiciones lineales (memoria plana).
- Aplicar transformaciones simples e introducir una convolución básica.
- Resolver pequeños retos guiados con comprobaciones automáticas.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Aumentar DPI para mejor visualización en Colab/local
plt.rcParams['figure.dpi'] = 120

def show(img, title=None):
    plt.figure(figsize=(5,3))
    if img.ndim == 2:
        plt.imshow(img, cmap='gray', vmin=img.min(), vmax=img.max())
    else:
        plt.imshow(img)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()
    
print("Entorno listo ✔")


## 1) Crear una imagen sintética
Crearemos una imagen RGB `img` de tamaño `H=240, W=360` con:
- Un **degradado** horizontal en el canal **R**.
- Un **degradado** vertical en el canal **G**.
- Un **círculo** central blanco añadido en los tres canales.


In [ ]:

H, W = 240, 360
x = np.linspace(0, 1, W)
y = np.linspace(0, 1, H)
X, Y = np.meshgrid(x, y)

R = (X * 255).astype(np.uint8)
G = (Y * 255).astype(np.uint8)
B = np.zeros_like(R, dtype=np.uint8)

# Círculo
cx, cy, r = W//2, H//2, 60
YY, XX = np.ogrid[:H, :W]
mask = (XX - cx)**2 + (YY - cy)**2 <= r**2
for C in (R, G, B):
    C[mask] = 255

img = np.stack([R, G, B], axis=2)  # HWC
print("img.shape =", img.shape, "| dtype =", img.dtype)
show(img, "Imagen sintética RGB (HWC)")


### 2) Información básica y visualización
Completa las celdas para inspeccionar dimensiones, tipo y valores mínimo/máximo por canal.


In [ ]:

# TODO: imprime (shape, dtype) y min/max por canal
print("Shape:", img.shape)
print("Dtype:", img.dtype)
print("Canales (R,G,B) min/max:",
      img[...,0].min(), img[...,0].max(),
      img[...,1].min(), img[...,1].max(),
      img[...,2].min(), img[...,2].max())


## 3) Indexación y *slicing*
**3.1** Extrae una **ventana** central de 120x180 píxeles y muéstrala.  
**3.2** Invierte la imagen verticalmente e **invertida horizontalmente**.  


In [ ]:

# 3.1 Ventana central 120x180
h, w = 120, 180
y0 = (H - h)//2; y1 = y0 + h
x0 = (W - w)//2; x1 = x0 + w
img_crop = img[y0:y1, x0:x1, :]
show(img_crop, "Recorte central 120x180")

# 3.2 Inversiones
img_flip_v = img[::-1, :, :]
img_flip_h = img[:, ::-1, :]
show(img_flip_v, "Invertida vertical")
show(img_flip_h, "Invertida horizontal")


## 4) Canales RGB
**4.1** Muestra por separado cada canal en escala de grises.  
**4.2** Crea una imagen **solo canal R** (los otros a 0).  


In [ ]:

# 4.1 Canales en escala de grises
show(img[...,0], "Canal R")
show(img[...,1], "Canal G")
show(img[...,2], "Canal B")

# 4.2 Solo canal R
only_R = np.zeros_like(img)
only_R[...,0] = img[...,0]
show(only_R, "Solo canal R (G=B=0)")


## 5) Normalización de intensidades
Convierte `img` a `float32` y normaliza en `[0,1]`. Comprueba el rango resultante.


In [ ]:

img_f = img.astype(np.float32) / 255.0
print(img_f.dtype, img_f.min(), img_f.max())
show(img_f, "Imagen normalizada [0,1]")


## 6) HWC vs CHW y posición lineal
**6.1** Convierte `img` de HWC a CHW y comprueba la nueva `shape`.  
**6.2** Para `W=360, H=240, C=3`, calcula la **posición lineal** del píxel `(x=15, y=192, c=2)` en memoria plana CHW usando:
```
index = x + y*W + c*W*H
```


In [ ]:

img_chw = np.transpose(img, (2,0,1))  # C,H,W
print("HWC -> CHW:", img.shape, "->", img_chw.shape)

W_, H_, C_ = W, H, 3
x, y, c = 15, 192, 2
index = x + y*W_ + c*W_*H_
print("Índice lineal (x=15,y=192,c=2):", index)


## 7) Reescalado simple (interpolación vecindad cercana)
Implementa un **re-escalado 2x** por vecindad cercana, sin usar librerías externas.


In [ ]:

scale = 2
H2, W2 = H*scale, W*scale
up = np.zeros((H2, W2, 3), dtype=img.dtype)
for yy in range(H2):
    for xx in range(W2):
        y_src = yy // scale
        x_src = xx // scale
        up[yy, xx] = img[y_src, x_src]
show(up, "Reescalado 2x (Nearest-Neighbor)")


## 8) Convolución básica (difuminado)
Aplica un **filtro de media 3x3** al canal R. Evita dependencias externas.


In [ ]:

K = np.ones((3,3), dtype=np.float32) / 9.0
R = img[...,0].astype(np.float32)
Hk, Wk = R.shape
out = np.zeros_like(R)
for i in range(1, Hk-1):
    for j in range(1, Wk-1):
        patch = R[i-1:i+2, j-1:j+2]
        out[i,j] = np.sum(patch * K)
out = np.clip(out, 0, 255).astype(np.uint8)

blur = img.copy()
blur[...,0] = out
show(blur, "Canal R difuminado (media 3x3)")


## 9) Tamaño en memoria
Calcula el tamaño (en bytes y MB) de una imagen RGB `1920x1080` de 8 bits por canal.


In [ ]:

Hq, Wq, Cq = 1080, 1920, 3
bytes_total = Hq * Wq * Cq * 1  # 1 byte por canal (8 bits)
print("Bytes:", bytes_total, "| MB aprox:", bytes_total / (1024*1024))


## 10) Retos rápidos
1. Crea una función `mirror_quadrants(img)` que **intercambie** cuadrantes de la imagen (arriba-izquierda con abajo-derecha, etc.).  
2. Implementa `to_grayscale(img)` que convierta RGB a escala de grises usando media simple por canal (sin usar librerías).  
3. Escribe `histogram_channel(img, c)` que devuelva un histograma (array de longitud 256) para el canal `c`.


In [ ]:

def mirror_quadrants(im):
    H, W, C = im.shape
    h2, w2 = H//2, W//2
    out = im.copy()
    # Intercambio de cuadrantes: TL <-> BR, TR <-> BL
    out[:h2,:w2], out[h2:,w2:] = im[h2:,w2:].copy(), im[:h2,:w2].copy()
    out[:h2,w2:], out[h2:,:w2] = im[h2:,:w2].copy(), im[:h2,w2:].copy()
    return out

def to_grayscale(im):
    g = im[...,0].astype(np.float32) + im[...,1].astype(np.float32) + im[...,2].astype(np.float32)
    g = (g / 3.0).astype(np.uint8)
    return g

def histogram_channel(im, c):
    h = np.zeros(256, dtype=np.int32)
    vals = im[...,c].ravel()
    for v in vals:
        h[v] += 1
    return h

# Pruebas rápidas
show(mirror_quadrants(img), "Cuadrantes espejados")
show(to_grayscale(img), "Escala de grises (media)")

histR = histogram_channel(img, 0)
print("Suma histograma R:", histR.sum(), "==", img.shape[0]*img.shape[1])


---
### Siguientes pasos
En la próxima práctica:
- Convoluciones con distintos kernels (bordes, nitidez).
- Reescalado bilineal y rotaciones.
- Preparación de tensores para modelos (normalización estándar, *data augmentation*).
